In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
#import xgboost as xgb
from scipy.sparse import vstack, csr_matrix, save_npz, load_npz
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import StratifiedKFold
#from sklearn.metrics import roc_auc_score
import gc
from functools import cmp_to_key
import time
import matplotlib.pyplot as plt
from numba import jit
from functools import reduce
gc.enable()

dtypes = {
        'MachineIdentifier':                                    'category',
        'ProductName':                                          'category',
        'EngineVersion':                                        'category',
        'AppVersion':                                           'category',
        'AvSigVersion':                                         'category',
        'IsBeta':                                               'int8',
        'RtpStateBitfield':                                     'float16',
        'IsSxsPassiveMode':                                     'int8',
        'DefaultBrowsersIdentifier':                            'float16',
        'AVProductStatesIdentifier':                            'float32',
        'AVProductsInstalled':                                  'float16',
        'AVProductsEnabled':                                    'float16',
        'HasTpm':                                               'int8',
        'CountryIdentifier':                                    'int16',
        'CityIdentifier':                                       'float32',
        'OrganizationIdentifier':                               'float16',
        'GeoNameIdentifier':                                    'float16',
        'LocaleEnglishNameIdentifier':                          'int8',
        'Platform':                                             'category',
        'Processor':                                            'category',
        'OsVer':                                                'category',
        'OsBuild':                                              'int16',
        'OsSuite':                                              'int16',
        'OsPlatformSubRelease':                                 'category',
        'OsBuildLab':                                           'category',
        'SkuEdition':                                           'category',
        'IsProtected':                                          'float16',
        'AutoSampleOptIn':                                      'int8',
        'PuaMode':                                              'category',
        'SMode':                                                'float16',
        'IeVerIdentifier':                                      'float16',
        'SmartScreen':                                          'category',
        'Firewall':                                             'float16',
        'UacLuaenable':                                         'float32',
        'Census_MDC2FormFactor':                                'category',
        'Census_DeviceFamily':                                  'category',
        'Census_OEMNameIdentifier':                             'float16',
        'Census_OEMModelIdentifier':                            'float32',
        'Census_ProcessorCoreCount':                            'float16',
        'Census_ProcessorManufacturerIdentifier':               'float16',
        'Census_ProcessorModelIdentifier':                      'float16',
        'Census_ProcessorClass':                                'category',
        'Census_PrimaryDiskTotalCapacity':                      'float32',
        'Census_PrimaryDiskTypeName':                           'category',
        'Census_SystemVolumeTotalCapacity':                     'float32',
        'Census_HasOpticalDiskDrive':                           'int8',
        'Census_TotalPhysicalRAM':                              'float32',
        'Census_ChassisTypeName':                               'category',
        'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float16',
        'Census_InternalPrimaryDisplayResolutionHorizontal':    'float16',
        'Census_InternalPrimaryDisplayResolutionVertical':      'float16',
        'Census_PowerPlatformRoleName':                         'category',
        'Census_InternalBatteryType':                           'category',
        'Census_InternalBatteryNumberOfCharges':                'float32',
        'Census_OSVersion':                                     'category',
        'Census_OSArchitecture':                                'category',
        'Census_OSBranch':                                      'category',
        'Census_OSBuildNumber':                                 'int16',
        'Census_OSBuildRevision':                               'int32',
        'Census_OSEdition':                                     'category',
        'Census_OSSkuName':                                     'category',
        'Census_OSInstallTypeName':                             'category',
        'Census_OSInstallLanguageIdentifier':                   'float16',
        'Census_OSUILocaleIdentifier':                          'int16',
        'Census_OSWUAutoUpdateOptionsName':                     'category',
        'Census_IsPortableOperatingSystem':                     'int8',
        'Census_GenuineStateName':                              'category',
        'Census_ActivationChannel':                             'category',
        'Census_IsFlightingInternal':                           'float16',
        'Census_IsFlightsDisabled':                             'float16',
        'Census_FlightRing':                                    'category',
        'Census_ThresholdOptIn':                                'float16',
        'Census_FirmwareManufacturerIdentifier':                'float16',
        'Census_FirmwareVersionIdentifier':                     'float32',
        'Census_IsSecureBootEnabled':                           'int8',
        'Census_IsWIMBootEnabled':                              'float16',
        'Census_IsVirtualDevice':                               'float16',
        'Census_IsTouchEnabled':                                'int8',
        'Census_IsPenCapable':                                  'int8',
        'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
        'Wdft_IsGamer':                                         'float16',
        'Wdft_RegionIdentifier':                                'float16',
        'HasDetections':                                        'int8'
        }


In [2]:
print('Download Train and Test Data.\n')
%time train = pd.read_csv('/home/ryan/cs/datasets/microsoft/train.csv', dtype=dtypes, low_memory=True)
train['MachineIdentifier'] = train.index.astype('uint32')
%time test  = pd.read_csv('/home/ryan/cs/datasets/microsoft/test.csv',  dtype=dtypes, low_memory=True)
test['MachineIdentifier']  = test.index.astype('uint32')
osver_timestamps = np.load('/home/ryan/cs/datasets/microsoft/OSVersionTimestamps.npy')[()]
avsig_timestamps = np.load('/home/ryan/cs/datasets/microsoft/AvSigVersionTimestamps.npy')[()]
if 5244810 in train.index:
    train.loc[5244810,'AvSigVersion'] = '1.273.1144.0'
    train['AvSigVersion'].cat.remove_categories('1.2&#x17;3.1144.0',inplace=True)
    
gc.collect()

Download Train and Test Data.

CPU times: user 1min 14s, sys: 2.31 s, total: 1min 16s
Wall time: 1min 17s
CPU times: user 1min 9s, sys: 1.77 s, total: 1min 11s
Wall time: 1min 12s


201019

In [6]:
columns_to_delete = ['HasDetections', #label
                     'MachineIdentifier',
                     'DefaultBrowsersIdentifier', #nan columns
                     'PuaMode',
                     'ProductName', #imbalanced columns
                     'IsBeta',
                     'IsSxsPassiveMode',
                     'HasTpm',
                     'AutoSampleOptIn',
                     'PuaMode',
                     'UacLuaenable',
                     'Census_DeviceFamily',
                     'Census_ProcessorClass',
                     'Census_IsPortableOperatingSystem',
                     'Census_IsFlightsDisabled',
                     'Census_IsVirtualDevice',
                     'Census_OSSkuName', # overlap columns
                     'OsVer',
                     'Census_OSArchitecture',
                     'Census_OSInstallLanguageIdentifier'
                     'Census_InternalBatteryNumberOfCharges' # strange values
                    ]


columns_to_process = []
for col in train.columns:
    if col not in columns_to_delete:
        columns_to_process.append(col)

In [30]:
def condense_feature(df, col, category_map, fill_with=None):
    feature = df[col]
    reverse_map = {}
    default_val = None
    categories = feature.unique()
    categories = categories[~pd.isnull(categories)]
    
    for k, v_arr in category_map.items():
        if len(v_arr) != 0:
            for v in v_arr:       
                reverse_map[v] = k
                index = np.argwhere(categories==v)
                categories = np.delete(categories, index)         
        else:
            default_val = k
            
    for v in categories:
        reverse_map[v] = default_val
        
    condensed = df[col].map(reverse_map)
    if fill_with:
        condensed = condensed.fillna(fill_with)
    else:
        condensed = condensed.fillna(default_val)
    
    return condensed

def generate_count_feature(df, col, counts=None):
    if counts is None:
        feature = df[col]
        groups = df.groupby(col)
        counts = groups.size()
    count_feature = df[col].map(counts)
    
    return count_feature, counts

def group_battery(x):
    x = x.lower()
    if ('li' in x) or ('ion' in x):
        return 1
    else:
        return 0
    
def generate_freq_feature(df, col, on='HasDetections', frequencies=None):
#     print(frequencies)
    if frequencies is None:
        groups = df.groupby(col)
        sizes = groups.size()
        sums = groups.sum()[on]
        frequencies = sums/sizes
    freq_feature = df[col].map(frequencies)
    
    return freq_feature, frequencies

def generate_version_mapping(df, col, num_splits=4, fill_val='0.0.0.0'):
    feature = df[col].astype(str)
    feature.fillna(fill_val)
    versions = feature.unique()
    
    def version_compare(x, y):
        x_splits = x.split('.')
        y_splits = y.split('.')
        for x_val, y_val in zip(x_splits, y_splits):
            try:
                int_x = int(x_val)
            except:
                int_x = 0
            try:
                int_y = int(y_val)
            except:
                int_y = 0
            if int_x > int_y:
                return 1
            elif int_x < int_y:
                return -1
        return 0

    sorted_versions = sorted(versions, key=cmp_to_key(version_compare))
    mapping = {}
    for i, v in enumerate(sorted_versions):
        mapping[v] = i
        
    return feature.map(mapping)

def generate_split_version(df, col, num_splits=4):
    feature = df[col].astype(str)
    sv = pd.DataFrame()
    for i in range(num_splits):
        sv[col + '_' + str(i)] = feature.apply(lambda x: x.split('.')[i]).astype(np.int16)
        
    return sv

def create_age_feature(df, col, raw_timestamps):
    timestamps = {}
    for k, v in raw_timestamps.items():
        timestamps[k] = v.toordinal()
    maxval = max(list(timestamps.values()))
    for k, v in timestamps.items():
        timestamps[k] = maxval - timestamps[k]
        
    return df[col].map(timestamps)

In [49]:
def generate_count_df(df, cols, counts_map={}):
    count_df = pd.DataFrame()
    for col in cols:
        if col in counts_map:
            count_df[col + '_counts'], _ = generate_count_feature(df, col, counts=counts_map[col])
        else:
            count_df[col + '_counts'], counts_map[col] = generate_count_feature(df, col)
        
    return count_df, counts_map

def generate_two_count_df(df1, df2, cols, counts_map={}):
    count_df1 = pd.DataFrame()
    count_df2 = pd.DataFrame()
    features = pd.concat([df1, df2])
    for col in cols:
        counts = features[col].value_counts()
        count_df1[col + '_counts'] = df1[col].map(counts)
        count_df2[col + '_counts'] = df2[col].map(counts)
        
    return count_df1, count_df2

        
def generate_freq_df(df, cols, freqs_map={}):
    freq_df = pd.DataFrame()
    for col in cols:
        print('Generating frequency feature for ' + col + '...')
        if col in freqs_map:
            freq_df[col + '_freqs'], _ = generate_freq_feature(df, col, frequencies=freqs_map[col])
        else:
            freq_df[col + '_freqs'],  freqs_map[col] =  generate_freq_feature(df, col)
            
    return freq_df, freqs_map

def generate_category_df(df, cols):
    cat_df = pd.DataFrame()
    for col in cols:
        cat_df[col+'_cat'] = df[col].astype('category')
        
    return cat_df
        
def generate_boolean_df(df, cols):
    bool_df = pd.DataFrame()
    for col in cols:
        bool_df[col+'_bool'] = df[col].astype(np.bool)
        
    return bool_df
        
def generate_gb_df(df, cols):
    gb_df = pd.DataFrame()
    for col in cols:
        gb_df[col+'_gb'] = df[col].fillna(df[col].median()).apply(lambda x: int(x/1000))
        
    return gb_df


In [59]:
def trim_all(df_train, df_test, columns_to_trim, delete_time_sensitive=False, delete_nan_thresh=0.8, fill='new'):
    fe_train = pd.DataFrame()
    fe_test = pd.DataFrame()
    for i, col in enumerate(columns_to_trim):
        print(str(i) + '/' + str(len(columns_to_trim)) + ' Trimming ' + col + '...')
        if False:
#         if col == 'AvSigVersion':
            fe_train[col] = df_train[col]
            fe_test[col] = df_test[col]
        else:
            tr_col, te_col, le_name_mapping = trim(df_train, df_test, col, fill='new')
            if delete_time_sensitive and 'nan' in le_name_mapping:
                percent = (te_col.values == le_name_mapping['nan']).sum() / float(len(te_col))
                if percent < delete_nan_thresh:
                    fe_train[col], fe_test[col] =  tr_col, te_col
                    
                else:
                    print('Deleted ' + col)
            else:
                fe_train[col], fe_test[col] =  tr_col, te_col
                    
        
    print('Done!')
    return fe_train, fe_test

def trim(raw_tr, raw_te, col, fill='new', factor=4, min_obs=1000):
#     raw_tr = raw_tr[:100]
#     raw_te = raw_te[:100]
    agg_tr = raw_tr.groupby(col).size().to_frame('tr_counts').reset_index()
    agg_te = raw_te.groupby(col).size().to_frame('te_counts').reset_index()
    
    agg = pd.merge(agg_tr, agg_te, on=col, how='outer')

    try:
        agg = agg.fillna(0)
    except:
        pass

    agg = agg[agg['tr_counts'] > min_obs].reset_index(drop=True)
    agg['total_counts'] = agg['tr_counts'] + agg['te_counts']
    agg = agg[(agg['tr_counts'] / agg['total_counts'] > (1.0/factor)) & (agg['tr_counts'] / agg['total_counts'] < ((factor-1.0)/factor))]
    agg[col+'_copy'] = agg[col]

    trim_tr = pd.merge(raw_tr[[col]], agg[[col, col+'_copy']], on=col, how='left')[col+'_copy'].astype(str)
    trim_te = pd.merge(raw_te[[col]], agg[[col, col+'_copy']], on=col, how='left')[col+'_copy'].astype(str)
    
    if fill == 'new':
        le = LabelEncoder().fit(trim_tr.values)
        mapping = dict(zip(le.classes_, le.transform(le.classes_)))
        trim_tr = pd.Series(le.transform(trim_tr))
        trim_te = pd.Series(le.transform(trim_te))
        trim_tr = trim_tr.astype('category').rename(col+'_ztrim')
        trim_te = trim_te.astype('category').rename(col+'_ztrim')
    
    elif fill == 'ordinal':
        # should be same uniques
        mapping = ordinal_mapping(trim_tr.values)
        good_vals = []
        for g in mapping.keys():
            if g != 'nan':
                good_vals.append(float(g))
        good_vals.sort()

        trim_tr = fill_ordinal(raw_tr[col], trim_tr, good_vals).astype(float).rename(col+'_otrim')
        trim_te = fill_ordinal(raw_te[col], trim_te, good_vals).astype(float).rename(col+'_otrim')
        
        trim_tr = trim_tr.map(mapping)
        trim_te = trim_te.map(mapping)
                
    else:
        raise Exception('fill should be new or ordinal')
    
    return trim_tr, trim_te, mapping


def fill_ordinal(raw_series, series, good_vals):

    curr_index = 0
    len_good_vals = len(good_vals)
    
    sorted_series = series.argsort()

    for idx in sorted_series:
        
        if series[idx] == 'nan':
            if curr_index >= len_good_vals - 1:
                series[idx] = good_vals[-1]
            elif raw_series.iloc[idx] < good_vals[curr_index+1]:
                series[idx] = good_vals[curr_index]
            else:
                series[idx] = good_vals[curr_index+1]
                curr_index += 1
    return series


def ordinal_mapping(vals, fill_val=-1):
    feature = vals.astype(float)
    feature[np.isnan(feature)] = fill_val

    versions = np.unique(feature)
    
    def version_compare(x, y):

        try:
            int_x = int(x_val)
        except:
            int_x = 0
        try:
            int_y = int(y_val)
        except:
            int_y = 0
        if int_x > int_y:
            return 1
        elif int_x < int_y:
            return -1
        return 0

    sorted_versions = sorted(versions, key=cmp_to_key(version_compare))
    mapping = dict(zip(sorted_versions, range(len(sorted_versions))))
        
    return mapping
    

In [33]:
# input raw df before trim: counts, condense, split_version, boolean, and non-categorical
def feature_engineer_1(df):

    fe = pd.DataFrame()
    
    fe['MoreThanOneAV'] = condense_feature(df, 'AVProductsInstalled', {False: [0.0, 1.0], True: []}, fill_with=False).astype(bool)
    fe['SkuCondensed'] = condense_feature(df, 'SkuEdition', {'Home': ['Home'], 'Pro': ['Pro'], 'SkuEtc':[]}).astype('category')
    fe['SmartScreen_condensed'] = condense_feature(df, 'SmartScreen', 
                                            {True: ['RequireAdmin', 'On', 'Warn', 'on', 'Enabled', 'warn', 'Block', 'Prompt', 'Promp', 'requireadmin', 'prompt', 'requireAdmin'], 
                                             False: []}).astype(np.bool)
    
    fe['BatteryType'] = df['Census_InternalBatteryType'].apply(group_battery)
    
    fe['IeVerIdentifier_num'] = df['IeVerIdentifier'].fillna(0.0).astype(np.int16)
    fe['ProcessorCoreCount_num'] = df['Census_ProcessorCoreCount'].fillna(4.0).astype(np.int16)
    fe['Census_ProcessorModelIdentifier_num'] = df['Census_ProcessorModelIdentifier'].fillna(0.0).astype(np.int16)
    fe['Census_InternalPrimaryDiagonalDisplaySizeInInches_num'] = df['Census_InternalPrimaryDiagonalDisplaySizeInInches'].fillna(15.5).astype(np.int16)
    
    fe['MachineCost'] = df['Census_ProcessorCoreCount']**2 * 20 + df['Census_SystemVolumeTotalCapacity']/20000 \
                        + df['Census_TotalPhyscialRAM']/200
    
    fe['AvSigAge'] = create_age_feature(df, 'AvSigVersion', avsig_timestamps)
    fe['OsVerAge'] = create_age_feature(df, 'Census_OSVersion', osver_timestamps)
    
    split_cols = ['Census_OSVersion', 'AppVersion', 'EngineVersion', 'AvSigVersion']
    
    gb_cols = ['Census_PrimaryDiskTotalCapacity', 'Census_SystemVolumeTotalCapacity', 'Census_TotalPhysicalRAM']
    
    for split_col in split_cols:
        fe = fe.join(generate_split_version(df, split_col, num_splits=4))
    

    
    fe = fe.join(generate_gb_df(df, gb_cols))
    
    return fe

# input trimmed df
def feature_engineer_2(df, frequencies_map={}):
    
    fe = pd.DataFrame()
    
    fe['EngineVersionMapped'] = generate_version_mapping(df, 'EngineVersion')
    fe['AppVersionMapped'] = generate_version_mapping(df, 'AppVersion')
    fe['AvSigVersionMapped'] = generate_version_mapping(df, 'AvSigVersion')
    
    freq_cols = ['OrganizationIdentifier', 'CountryIdentifier', 'IeVerIdentifier', 
                 'Census_ActivationChannel', 'Census_FirmwareManufacturerIdentifier', 
                 'Census_OSUILocaleIdentifier']
    
    freq_df, frequencies_map = generate_freq_df(df, freq_cols, freqs_map=frequencies_map)
    fe = fe.join(freq_df)
    
    # cost feature
    
    return fe, frequencies_map
    

In [28]:
print('Feature engineering train features...')
fe1_train = feature_engineer_1(train)
print('Feature engineering test features...')
fe1_test = feature_engineer_1(test)

Feature engineering train features...
Feature engineering test features...


In [42]:
# count_cols = ['EngineVersion', 'AppVersion', 'AvSigVersion', 'AVProductStatesIdentifier',
#               'CountryIdentifier', 'Census_OSVersion', 'CityIdentifier', 'OrganizationIdentifier', 
#               'GeoNameIdentifier', 'OsBuild', 
#               'OsBuildLab', 'IeVerIdentifier', 'Census_MDC2FormFactor', 'Census_OEMNameIdentifier', 
#               'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier', 
#               'Census_ProcessorModelIdentifier', 'Census_PrimaryDiskTypeName', 'Census_ChassisTypeName', 
#               'Census_PowerPlatformRoleName', 'Census_OSBranch', 'Census_OSBuildNumber', 
#               'Census_OSBuildRevision','Census_OSEdition','Census_OSInstallTypeName', 
#               'Census_OSUILocaleIdentifier', 'Census_ActivationChannel','Census_FirmwareManufacturerIdentifier',
#               'Census_FirmwareVersionIdentifier', 'Census_IsTouchEnabled']

count_cols = ['CityIdentifier', 'AvSigVersion', 'Census_FirmwareVersionIdentifier', 'Census_ProcessorModelIdentifier',
              'CountryIdentifier', 'GeoNameIdentifier', 'Census_OSVersion', 'Census_OSBuildRevision', 'Census_OEMNameIdentifier',
              'AVProductStatesIdentifier', 'Census_OSUILocaleIdentifier']

fe_count_cols = ['AppVersion_1', 'AppVersion_2', 'AppVersion_3', 
                 'Census_OSVersion_3', 'EngineVersion_2', 'EngineVersion_3', 
                 'AvSigVersion_1', 'AvSigVersion_2']

count_df1_train, count_df1_test = generate_two_count_df(train, test, count_cols)
count_df2_train, count_df2_test = generate_two_count_df(fe1_train, fe1_test, fe_count_cols)

In [46]:
count_df1_train = count_df1_train.fillna(0).astype(np.int32)
count_df1_test = count_df1_test.fillna(0).astype(np.int32)

In [64]:
non_categorical_columns = ['ProcessorCoreCount', 'Census_InternalPrimaryDiagonalDisplaySizeInInches', 
                           'Census_PrimaryDiskTotalCapacity', 'Census_SystemVolumeTotalCapacity', 
                           'Census_TotalPhysicalRAM']

columns_to_trim = []
for col in columns_to_process:
    if col not in non_categorical_columns:
        columns_to_trim.append(col)

%time trim_train, trim_test = trim_all(train, test, columns_to_trim)

0/61 Trimming EngineVersion...
1/61 Trimming AppVersion...
2/61 Trimming AvSigVersion...
3/61 Trimming RtpStateBitfield...
4/61 Trimming AVProductStatesIdentifier...
5/61 Trimming AVProductsInstalled...
6/61 Trimming AVProductsEnabled...
7/61 Trimming CountryIdentifier...
8/61 Trimming CityIdentifier...
9/61 Trimming OrganizationIdentifier...
10/61 Trimming GeoNameIdentifier...
11/61 Trimming LocaleEnglishNameIdentifier...
12/61 Trimming Platform...
13/61 Trimming Processor...
14/61 Trimming OsBuild...
15/61 Trimming OsSuite...
16/61 Trimming OsPlatformSubRelease...
17/61 Trimming OsBuildLab...
18/61 Trimming SkuEdition...
19/61 Trimming IsProtected...
20/61 Trimming SMode...
21/61 Trimming IeVerIdentifier...
22/61 Trimming SmartScreen...
23/61 Trimming Firewall...
24/61 Trimming Census_MDC2FormFactor...
25/61 Trimming Census_OEMNameIdentifier...
26/61 Trimming Census_OEMModelIdentifier...
27/61 Trimming Census_ProcessorCoreCount...
28/61 Trimming Census_ProcessorManufacturerIdentifier

In [65]:
# ordinal_columns = ['AppVersion_1',]
#                    'AppVersion_2', 'AppVersion_3',  
#                    'Census_OSVersion_3', 'EngineVersion_2']
ordinal_columns = ['AvSigVersion_2', 'Census_OSVersion_3']
ord_train = pd.DataFrame()
ord_test = pd.DataFrame()
for col in ordinal_columns:
    print('Generating ordinal trim column for ' + col + '...')
    ord_train_col, ord_test_col, _ = trim(fe1_train, fe1_test, col, fill='ordinal')
    ord_train[col+'_ord'] = ord_train_col
    ord_test[col+'_ord'] = ord_test_col
print('Done!')

Generating ordinal trim column for AvSigVersion_2...
Generating ordinal trim column for Census_OSVersion_3...
Done!


In [66]:
trim_train['HasDetections'] = train['HasDetections']

In [67]:
print('Feature engineering train features (part 2)...')
fe2_train, freqs = feature_engineer_2(trim_train)
print('Feature engineering test features (part 2)...')
fe2_test, _ = feature_engineer_2(trim_test, frequencies_map=freqs)

Feature engineering train features (part 2)...
Generating frequency feature for OrganizationIdentifier...
Generating frequency feature for CountryIdentifier...
Generating frequency feature for IeVerIdentifier...
Generating frequency feature for Census_ActivationChannel...
Generating frequency feature for Census_FirmwareManufacturerIdentifier...
Generating frequency feature for Census_OSUILocaleIdentifier...
Feature engineering test features (part 2)...
Generating frequency feature for OrganizationIdentifier...
Generating frequency feature for CountryIdentifier...
Generating frequency feature for IeVerIdentifier...
Generating frequency feature for Census_ActivationChannel...
Generating frequency feature for Census_FirmwareManufacturerIdentifier...
Generating frequency feature for Census_OSUILocaleIdentifier...


In [68]:
trim_train = trim_train.drop('HasDetections', axis=1)

In [96]:
# join fe1, fe2, and trimmed
train_final = reduce(lambda left, right: left.join(right, rsuffix='_oops'), [trim_train, ord_train, fe1_train, fe2_train, count_df1_train, count_df2_train])

test_final = reduce(lambda left, right: left.join(right, rsuffix='_oops'), [trim_test, ord_test, fe1_test, fe2_test, count_df1_test, count_df2_test])


In [120]:
len(trim_train.columns)

61

In [121]:
len(trim_test.columns)

61

In [105]:
for i in train_final.columns:
    if train_final[i].isnull().values.any():
        train_final[i] = train_final[i].fillna(0)
    if test_final[i].isnull().values.any():
        test_final[i] = test_final[i].fillna(0)
        
train_final = train_final.drop(['SkuCondensed'], axis=1)
test_final = test_final.drop(['SkuCondensed'], axis=1)    

In [71]:
train_final.head()

,EngineVersion,AppVersion,AvSigVersion,RtpStateBitfield,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,CountryIdentifier,CityIdentifier,OrganizationIdentifier,...,AVProductStatesIdentifier_counts,Census_OSUILocaleIdentifier_counts,AppVersion_1_counts,AppVersion_2_counts,AppVersion_3_counts,Census_OSVersion_3_counts,EngineVersion_2_counts,EngineVersion_3_counts,AvSigVersion_1_counts,AvSigVersion_2_counts
0,28,46,117,4,137,0,1,115,214,3,...,11333187,629790,11675668,5825662,5824824,959932,3833951,8049943,3887498,20517
1,20,16,7,4,137,0,1,177,415,3,...,11333187,5971350,1053108,1053048,488388,285590,299099,2695190,324819,243423
2,28,46,117,4,137,0,1,169,454,3,...,11333187,938464,11675668,5825662,5824824,959932,3833951,8049943,3887498,16282
3,28,46,117,4,137,0,1,171,566,19,...,11333187,156553,11675668,5825662,5824824,1565473,3833951,8049943,3887498,53752
4,28,46,117,4,137,0,1,76,710,19,...,11333187,5971350,11675668,5825662,5824824,252155,3833951,8049943,3887498,50449


In [106]:
train_final = train_final.drop(['AvSigVersion_3', 'Census_OSVersion_0', 'Census_OSVersion_1', 'EngineVersion_0',
                                'EngineVersion_1', 'AppVersion_0', 'Processor', 'AvSigVersion_0',
                                'SMode', 'Platform', 
                                'Census_IsPenCapable', 'Census_IsAlwaysOnAlwaysConnectedCapable',
                                 'Census_OSVersion_2', 'AvSigVersion_2_counts', 'AvSigVersion_counts', 'Census_OSVersion_counts'], axis=1)
test_final = test_final.drop(['AvSigVersion_3', 'Census_OSVersion_0', 'Census_OSVersion_1', 'EngineVersion_0',
                                'EngineVersion_1', 'AppVersion_0', 'Processor', 'AvSigVersion_0',
                                 'SMode', 'Platform', 
                               'Census_IsPenCapable', 'Census_IsAlwaysOnAlwaysConnectedCapable',
                                'Census_OSVersion_2', 'AvSigVersion_2_counts', 'AvSigVersion_counts', 'Census_OSVersion_counts'], axis=1)

ValueError: labels ['AvSigVersion_3' 'Census_OSVersion_0' 'Census_OSVersion_1'
 'EngineVersion_0' 'EngineVersion_1' 'AppVersion_0' 'Processor'
 'AvSigVersion_0' 'SMode' 'Platform' 'Census_IsPenCapable'
 'Census_IsAlwaysOnAlwaysConnectedCapable' 'Census_OSVersion_2'
 'AvSigVersion_2_counts' 'AvSigVersion_counts' 'Census_OSVersion_counts'] not contained in axis

In [67]:
save_df = False
if save_df:
    print('Saving train dataframe...')
    %time train_final.to_csv('train_df.csv')
    print('Saving test dataframe...')
    %time test_final.to_csv('test_df.csv')
    print('Done!')
    
load_df = False
if load_df:
    print('Loading train dataframe...')
    %time train_final = pd.read_csv('train_df.csv')
    print('Loading test dataframe...')
    %time test_final = pd.read_csv('test_df.csv')

In [75]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage(deep=True).sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [98]:
train_final = reduce_mem_usage(train_final)
test_final = reduce_mem_usage(test_final)

Mem. usage decreased to 1650.85 Mb (39.7% reduction)
Mem. usage decreased to 1453.21 Mb (39.7% reduction)


In [101]:
y_train = np.array(train['HasDetections'])
train_ids = train_final.index
test_ids  = test_final.index

gc.collect()

1847

In [78]:
@jit
def fast_auc(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    nfalse = 0
    auc = 0
    n = len(y_true)
    for i in range(n):
        y_i = y_true[i]
        nfalse += (1 - y_i)
        auc += y_i * nfalse
    auc /= (nfalse * (n - nfalse))
    return auc

def eval_auc(preds, dtrain):
    labels = dtrain.get_label()
    return 'auc', fast_auc(labels, preds), True

In [73]:
# #Fit OneHotEncoder
# ohe = OneHotEncoder(categories='auto', sparse=True, dtype='uint8').fit(train_final)

# #Transform data using small groups to reduce memory usage
# m = 100000
# train = vstack([ohe.transform(train_final[i*m:(i+1)*m]) for i in range(train_final.shape[0] // m + 1)])
# test  = vstack([ohe.transform(test_final[i*m:(i+1)*m])  for i in range(test_final.shape[0] // m +  1)])



In [126]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
skf.get_n_splits(train_ids, y_train)
feature_importances = []
fold_predictions = []
lgb_test_result  = np.zeros(test_ids.shape[0])
counter = 0

print('\nLightGBM\n')

for train_index, test_index in skf.split(train_ids, y_train):
    
    print('Fold {}\n'.
          format(counter + 1))
    X_fit = train_final.iloc[train_index]
    X_val = train_final.iloc[test_index]

    X_fit, X_val = csr_matrix(X_fit, dtype='float32'), csr_matrix(X_val, dtype='float32')
    y_fit, y_val = y_train[train_index], y_train[test_index]

    lgb_model = lgb.LGBMClassifier(max_depth=-1,
                                   n_estimators=30000,
                                   learning_rate=0.05,
                                   num_leaves=2**12-1,
                                   colsample_bytree=0.28,
                                   objective='binary', 
                                   n_jobs=-1)
                                   
    %time lgb_model.fit(X_fit, y_fit, eval_metric='auc', eval_set=[(X_val, y_val)], verbose=100, early_stopping_rounds=100)        

    feature_importances.append(lgb_model.feature_importances_)
    
    test_final_csr = csr_matrix(test_final, dtype='float32')
    fold_predictions.append(lgb_model.predict_proba(test_final_csr)[:,1])
    counter += 1
    
    gc.collect()



LightGBM

Fold 1

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.726758	valid_0's binary_logloss: 0.611818
[200]	valid_0's auc: 0.732514	valid_0's binary_logloss: 0.602729
[300]	valid_0's auc: 0.734379	valid_0's binary_logloss: 0.60033
[400]	valid_0's auc: 0.734947	valid_0's binary_logloss: 0.599608
[500]	valid_0's auc: 0.735135	valid_0's binary_logloss: 0.599334
[600]	valid_0's auc: 0.735112	valid_0's binary_logloss: 0.599318
Early stopping, best iteration is:
[546]	valid_0's auc: 0.735157	valid_0's binary_logloss: 0.599294
CPU times: user 1h 30min 32s, sys: 12.3 s, total: 1h 30min 45s
Wall time: 11min 48s
Fold 2

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's auc: 0.727014	valid_0's binary_logloss: 0.611806
[200]	valid_0's auc: 0.732733	valid_0's binary_logloss: 0.602716
[300]	valid_0's auc: 0.734565	valid_0's binary_logloss: 0.600347
[400]	valid_0's auc: 0.735148	valid_0's binary_logloss: 0.5996
[500]	valid_0's 

In [127]:
n_splits=5
predictions = reduce(lambda x, y: np.add(x, y), fold_predictions)

In [128]:
submission = pd.read_csv('/home/ryan/cs/datasets/microsoft/sample_submission.csv')
submission['HasDetections'] = predictions / n_splits
submission.to_csv('lgb_submission_my_trim_5.csv', index=False)

In [129]:
submission.head()

,MachineIdentifier,HasDetections
0,0000010489e3af074adeac69c53e555e,0.544161
1,00000176ac758d54827acd545b6315a5,0.601671
2,0000019dcefc128c2d4387c1273dae1d,0.658427
3,0000055553dc51b1295785415f1a224d,0.460735
4,00000574cefffeca83ec8adf9285b2bf,0.607968


In [138]:
fi = pd.DataFrame({'col':train_final.columns, 'importance':feature_importances[0]}, )
fi.sort_values('importance', axis=0, ascending=False)

,col,importance
53,Census_FirmwareVersionIdentifier,134785
8,CityIdentifier,130425
26,Census_OEMModelIdentifier,126613
29,Census_ProcessorModelIdentifier,108351
7,CountryIdentifier,107972
10,GeoNameIdentifier,97430
11,LocaleEnglishNameIdentifier,93590
38,Census_OSVersion,81978
25,Census_OEMNameIdentifier,81381
41,Census_OSBuildRevision,71350


In [135]:
len(train_final.columns)

103

In [133]:
len(feature_importances[0])

61